In [1]:
import pandas as pd
import os
import datetime
import pickle
import re
import time

## 1. Import articles with area codes

In [2]:
# Reg relevant articles
df_regSentsExpandRelevant=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegSentsExpand_NounChunks_Area.pkl')
print(df_regSentsExpandRelevant.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493418 entries, 0 to 493417
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           493418 non-null  int64 
 1   RegSentsExpand               493418 non-null  object
 2   NounChunksMatch              493418 non-null  int64 
 3   NounChunkMatchWords          493418 non-null  object
 4   NounChunkMatchWordsFiltered  493418 non-null  object
 5   NounChunkMatchFiltered       493418 non-null  int64 
 6   DominantNounChunk            493418 non-null  object
 7   DominantNounChunkArea        493418 non-null  object
 8   AllAreas                     493418 non-null  object
 9   AreaCount                    493418 non-null  object
 10  DominantArea                 493418 non-null  object
 11  AllDistinctAreas             493418 non-null  object
 12  UniqueDistinctAreas          493418 non-null  object
 13  DistinctAreaCo

In [3]:
# All news data
allNews=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml.pkl')
print(allNews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             822737 non-null  object        
 1   Title          822737 non-null  object        
 2   Type           822737 non-null  object        
 3   StartDate      822737 non-null  datetime64[ns]
 4   EndDate        822737 non-null  object        
 5   Text           822737 non-null  object        
 6   TextWordCount  822737 non-null  object        
 7   PubTitle       822737 non-null  object        
 8   SourceType     822737 non-null  object        
 9   Year           822737 non-null  int64         
 10  Month          822737 non-null  int64         
 11  Newspaper      822737 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 75.3+ MB
None


In [4]:
# Merge with all news data
allNews['ID']=allNews['ID'].astype('int64')
df=df_regSentsExpandRelevant.merge(allNews[['ID','Title','Type','SourceType','StartDate','Newspaper','Year','Month']],
                                  on='ID',how='left')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493418 entries, 0 to 493417
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           493418 non-null  int64         
 1   RegSentsExpand               493418 non-null  object        
 2   NounChunksMatch              493418 non-null  int64         
 3   NounChunkMatchWords          493418 non-null  object        
 4   NounChunkMatchWordsFiltered  493418 non-null  object        
 5   NounChunkMatchFiltered       493418 non-null  int64         
 6   DominantNounChunk            493418 non-null  object        
 7   DominantNounChunkArea        493418 non-null  object        
 8   AllAreas                     493418 non-null  object        
 9   AreaCount                    493418 non-null  object        
 10  DominantArea                 493418 non-null  object        
 11  AllDistinctAreas          

In [5]:
print(df['Type'].value_counts())
print(df['SourceType'].value_counts())

News    493418
Name: Type, dtype: int64
Newspapers    493418
Name: SourceType, dtype: int64


In [6]:
# # Refine to sourcetype=newspaper
# df=df[df['SourceType']=='Newspapers'].reset_index(drop=True)
# print(df.info())

In [7]:
print(df.head())

           ID                                     RegSentsExpand  \
0   292620682  Now, one hopes, more attention will be paid to...   
1   307490698  His economic program as a candidate in the Rep...   
2   307420103  As the Bank of New England hearings suggest, t...   
3   307576786  The Justice Department contends that the Eight...   
4  1944990599  "The remittances from drivers alone reach almo...   

   NounChunksMatch                                NounChunkMatchWords  \
0                8  [land use, civil right, land use, land use, lo...   
1                2                                 [tax cut, tax cut]   
2               10  [new england, bank hold company, new england, ...   
3                4  [civil action, criminal activity, civil claim,...   
4                1                            [government regulation]   

                         NounChunkMatchWordsFiltered  NounChunkMatchFiltered  \
0  [land use, civil right, land use, land use, pu...                    

## 2. Create dummies for areas

In [8]:
# List of columns for different approaches
area_range=15    # Number of areas + 1
col_list=[]
for i in range(1,area_range):
    var1='DominantNounChunkArea'+str(i)
    col_list.append(var1)
    var2='DominantArea'+str(i)
    col_list.append(var2)
    var3='UniqueDistinctArea'+str(i)
    col_list.append(var3)
    var4='DominantDistinctArea'+str(i)
    col_list.append(var4)

### 2.1 Dummies by dominant noun chunk area (dnca)

In [9]:
for i in range(1,area_range):
    var='DominantNounChunkArea'+str(i)
    df[var]=0
    for j in range(0, len(df)):
        if i in df['DominantNounChunkArea'][j]:
            df[var][j]=1
print(df)

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


                ID                                     RegSentsExpand  \
0        292620682  Now, one hopes, more attention will be paid to...   
1        307490698  His economic program as a candidate in the Rep...   
2        307420103  As the Bank of New England hearings suggest, t...   
3        307576786  The Justice Department contends that the Eight...   
4       1944990599  "The remittances from drivers alone reach almo...   
...            ...                                                ...   
493413  2009071770  • A Feb. 28 KidsPost article about an upcoming...   
493414  1933811831  The Washington Post is committed to correcting...   
493415  2411004924  Kohl's has reopened most of its more than 1,10...   
493416  1950787340  Hadges left the DEA in May to join the pharmac...   
493417  2321287809  The building does not have carbon monoxide det...   

        NounChunksMatch                                NounChunkMatchWords  \
0                     8  [land use, civil rig

### 2.2 Dummies by dominant area (da)

In [10]:
for i in range(1,area_range):
    var='DominantArea'+str(i)
    df[var]=0
    for j in range(0, len(df)):
        if i in df['DominantArea'][j]:
            df[var][j]=1

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### 2.3 Dummies by distinct area (uda)

In [11]:
for i in range(1,area_range):
    var='UniqueDistinctArea'+str(i)
    df[var]=0
    for j in range(0, len(df)):
        if i in df['UniqueDistinctAreas'][j]:
            df[var][j]=1
#print(df.info())

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### 2.4 Dummies by dominant distinct area (dda)

In [12]:
for i in range(1,area_range):
    var='DominantDistinctArea'+str(i)
    df[var]=0
    for j in range(0, len(df)):
        if i in df['DominantDistinctArea'][j]:
            df[var][j]=1
#print(df.info())

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 3. Monthly article counts by area

In [13]:
# Aggregate monthly article counts
monthlyAreaCount=df[['Newspaper','Year','Month']+col_list].groupby(['Newspaper','Year','Month']).agg('sum').reset_index()
print(monthlyAreaCount.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945 entries, 0 to 2944
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Newspaper                2945 non-null   object
 1   Year                     2945 non-null   int64 
 2   Month                    2945 non-null   int64 
 3   DominantNounChunkArea1   2945 non-null   int64 
 4   DominantArea1            2945 non-null   int64 
 5   UniqueDistinctArea1      2945 non-null   int64 
 6   DominantDistinctArea1    2945 non-null   int64 
 7   DominantNounChunkArea2   2945 non-null   int64 
 8   DominantArea2            2945 non-null   int64 
 9   UniqueDistinctArea2      2945 non-null   int64 
 10  DominantDistinctArea2    2945 non-null   int64 
 11  DominantNounChunkArea3   2945 non-null   int64 
 12  DominantArea3            2945 non-null   int64 
 13  UniqueDistinctArea3      2945 non-null   int64 
 14  DominantDistinctArea3    2945 non-null  

In [14]:
print(monthlyAreaCount.head())

      Newspaper  Year  Month  DominantNounChunkArea1  DominantArea1  \
0  Boston Globe  1985      1                      21             15   
1  Boston Globe  1985      2                      29             24   
2  Boston Globe  1985      3                      34             22   
3  Boston Globe  1985      4                      32             23   
4  Boston Globe  1985      5                      37             25   

   UniqueDistinctArea1  DominantDistinctArea1  DominantNounChunkArea2  \
0                   16                     10                      21   
1                   12                     12                      27   
2                   17                     13                      29   
3                   15                     12                      33   
4                   18                     16                      32   

   DominantArea2  UniqueDistinctArea2  ...  UniqueDistinctArea12  \
0             17                   13  ...                     4  

In [15]:
monthlyAreaCount.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_MonthlyArticleCountByNewspaper.csv',index=False)

## 4. Merge with sentiment scores

### 4.1 Article-level sentiment scores

In [16]:
# Merge with sentiment scores
sentimentScores=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/RegRelevant_ArticleSentimentScores.csv')
print(sentimentScores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493418 entries, 0 to 493417
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                493418 non-null  int64  
 1   StartDate         493418 non-null  object 
 2   Newspaper         493418 non-null  object 
 3   UncertaintyScore  493418 non-null  float64
 4   GIscore           493418 non-null  float64
 5   LMscore           493418 non-null  float64
 6   LSDscore          493418 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 26.4+ MB
None


In [17]:
# Merge
df['ID']=df['ID'].astype('int64')
df2=df.merge(sentimentScores[['ID','UncertaintyScore','GIscore','LMscore','LSDscore']],on='ID',how='left')
#print(df2.info())

In [18]:
df2[['ID','StartDate','Newspaper','UncertaintyScore','GIscore','LMscore','LSDscore']+col_list].to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_ArticleSentimentScores.csv',index=False)

### 4.2 Monthly average score by area

In [19]:
# Calculate average monthly scores by area
df_reg=df2
df_reg['ArticleCount']=1
monthlyAreaScores=df_reg[['Year','Month','ArticleCount']].\
    groupby(['Year','Month']).agg({'ArticleCount':'sum'}).reset_index()
print(monthlyAreaScores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Year          428 non-null    int64
 1   Month         428 non-null    int64
 2   ArticleCount  428 non-null    int64
dtypes: int64(3)
memory usage: 10.2 KB
None


In [20]:
for var in col_list:
    df_area=df_reg[df_reg[var]==1].reset_index(drop=True)
    areaScore=df_area[['Year','Month','UncertaintyScore','GIscore','LSDscore','LMscore','ArticleCount']].\
                                    groupby(['Year','Month']).agg({'UncertaintyScore':'mean','GIscore':'mean',
                                   'LSDscore':'mean','LMscore':'mean','ArticleCount':'sum'}).reset_index()
    areaScore=areaScore.rename(columns={'UncertaintyScore':'UncertaintyScore'+'_'+var,'GIscore':'GIscore'+'_'+var,
                                       'LSDscore':'LSDscore'+'_'+var,'LMscore':'LMscore'+'_'+var,'ArticleCount':'ArticleCount'+'_'+var})
    monthlyAreaScores=monthlyAreaScores.merge(areaScore,on=['Year','Month'],how='left')
print(monthlyAreaScores.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428 entries, 0 to 427
Columns: 283 entries, Year to ArticleCount_DominantDistinctArea14
dtypes: float64(224), int64(59)
memory usage: 949.6 KB
None


In [21]:
monthlyAreaScores.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_MonthlySentimentScores.csv', index=False)

## A1. Examine the results

In [22]:
# Investigate the results
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493418 entries, 0 to 493417
Data columns (total 83 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           493418 non-null  int64         
 1   RegSentsExpand               493418 non-null  object        
 2   NounChunksMatch              493418 non-null  int64         
 3   NounChunkMatchWords          493418 non-null  object        
 4   NounChunkMatchWordsFiltered  493418 non-null  object        
 5   NounChunkMatchFiltered       493418 non-null  int64         
 6   DominantNounChunk            493418 non-null  object        
 7   DominantNounChunkArea        493418 non-null  object        
 8   AllAreas                     493418 non-null  object        
 9   AreaCount                    493418 non-null  object        
 10  DominantArea                 493418 non-null  object        
 11  AllDistinctAreas          

In [23]:
# Most negative/positive articles in specified area
temp=df2[(df2['DominantDistinctArea3']==1) & (df2['Year']==2014) & (df2['Month']==10)]
temp=temp.sort_values('LMscore').reset_index(drop=True)
print(temp[['ID','LMscore']])

             ID    LMscore
0    1612189646 -14.285714
1    1615153643 -13.461538
2    1615196375 -13.461538
3    1615171897  -9.634551
4    1598947927  -9.459459
..          ...        ...
203  1617023940   1.785714
204  1618018985   2.272727
205  1610817937   2.739726
206  1566726761   2.739726
207  1586064778   2.777778

[208 rows x 2 columns]


In [24]:
for i in range(0,len(temp)):
    print(temp['ID'][i],temp['Title'][i],':',temp['Newspaper'][i],'\n',temp['RegSentsExpand'][i],temp['NounChunkMatchWordsFiltered'][i],
          temp['AllDistinctAreas'][i], temp['DominantDistinctArea'][i],temp['LMscore'][i],'\n')

1612189646 Class-Action Case Accuses G.M. of Neglect Beyond Ignition Switch : New York Times 
 Based on documents and communications with the federal safety regulators, the complaint cited a number of cases in which it contended that recalls for a documented safety problem were limited or late, or warranty claims were ignored -- as with recalls for axle shafts that could break and a defect in a shift cable that could cause cars to roll out of park and crash. That report was sharply critical of G.M.'s culture and described instances in which the company's engineers, product developers and legal department overlooked or failed to act on safety concerns. The complaint used a previously disclosed email to Ms. Barra to suggest the company's knowledge of safety problems stretched to the highest levels. " ['federal safety'] [3] [3] -14.285714285714285 

1615153643 What's News: Business & Finance : Wall Street Journal 
 An air-bag recall is turning into a new safety crisis for the auto industr

 While there is no national tally of how many people might be affected, brokers and insurers report that many individuals and small businesses renewed the old policies, if they had the choice, because they often cost less than new coverage. More than two dozen states allowed the plans to be renewed, although in some cases for only a year. In California, regulators required individual plans that fell short of the health law to be discontinued by the end of 2013 but allowed those offered by small businesses to continue through 2015. ['insurer report', 'small business', 'small business'] [3] [3] -1.0989010989010992 

1609440929 Montgomery official addresses cab and ride-share interests (Posted 2014-10-09 23:36:23) : The Washington Post 
 Barnes was treasurer of Berliner's first council campaign 2006, and also is a frequent contributor to local campaigns. Representatives for Uber did not attend the committee hearing, but sent written testimony signaling that the company was ready to work w

 Regulator: Pact With Lenders Could Expand Mortgage Access A top federal housing regulator said Fannie Mae, Freddie Mac and lenders had reached an agreement in principle that could expand access to mortgages for many Americans. Toyota Recalls 247,000 Vehicles Because Air Bags May Rupture Toyota Motor issued a recall for 247,000 vehicles that have an air bag that can rupture and send shrapnel into the passenger compartment, an expansion of an earlier recall. ISS, in Switch, Backs Chiquita-Fyffes Deal An influential shareholder advisory firm changed course and recommended Chiquita shareholders vote in favor of its planned tie-up with Irish fruit grower Fyffes, a potential blow to the Brazilian firms attempting to buy Chiquita. ['fannie mae', 'freddie mac', 'expand access', 'air bag', 'air bag'] [3, 3] [3] -0.8695652173913043 

1612767353 Goldman Earns Higher Profits, but Stock Falls on Worries Over Sustainable Returns : New York Times 
 It's about visibility and sustainability of returns

In [25]:
for i in range(0,len(df)):
    if  (df['Year'][i]==2002) & (df['Month'][i]==7):
        print(df['ID'][i],df['Title'][i])

409278502 Water Cleanup Overdue
409288669 DID YOU HEAR? . . .
409294241 Vouchers Fail the Test
409348878 It's the Ethics, Stupid
421716554 IN BRIEF; Also
421925759 IN BRIEF; Also
408846627 Related happenings Monday on Capitol Hill
421874710 Preview / WEEK OF JULY 15-21; Also
408881099 Plan aims to stop fraud in business ; Bush proposal seeks tougher enforcement
409307467 Key Lawmaker Calls For Probe of Analyst
405467634 NUMBERS SUPPORT CHARTER SCHOOLS DOE STATS SHOW PARITY IN DISTRICTS
405465701 RENT DEBATE CENTERS ON IMPACT OF DECONTROL
405459102 DEMOCRATS TARGET BUSH'S BUSINESS ETHICS ADS SEEK TO TURN RECORD AS CEO AGAINST PRESIDENT
398990959 Ahead of the Tape
398829272 Ahead of the Tape
419617211 Section 8 ownership plan gets into gear ; Section 8 option serves home buyers
432133174 World Briefing Europe: Britain: Suit In Bungled Embryo Case
419635249 Woman charged in overdose
421709306 The Region; Cities Aim to Shift Sand to Eroded Beaches; Conservation: Coalition seeks to streamli

398802971 Career Journal: Wanted: Ethical Employer --- Job Hunters, Seeking to Avoid An Enron or an Andersen, Find It Isn't Always Easy
421715405 The State; IN BRIEF / SACRAMENTO; Referendum Sought on Greenhouse Gases
409381001 For Cheney, Tarnish From Halliburton; Firm's Fall Raises Questions About Vice President's Leadership There
432124488 Metro Briefing New York: White Plains: Ephedra Ban Ordered
421918554 Los Angeles; Glendale Finds No Impropriety in Its Enron Deal
398807144 Unsecured-Creditors Committee Is Selected for WorldCom Inc.
409323797 Approva Corp. plans to hire si ...
399006907 Vivendi's 26% Slide Pressures European Markets
409344844 White Denies Allegations Of Price Manipulation; A Former Enron Executive, Army Chief Testifies in Senate
409293956 Conferees to Haggle With Eye on Nov.; Political Stakes High In Corporate Reform
409316234 Bush Took Oil Firm's Loans as Director; Practice Would Be Banned in President's New Corporate Abuse Policy
421893924 UC Irvine Business Et

421869662 IN BRIEF / FINANCE; Fed Questions Citigroup on Loans
421740709 Council to Study LAX Shooting Response; Violence: Agencies will report on their reaction amid calls to merge airport police, LAPD.
419625658 CLARK/BARDES INC.: Worries, reports prompt shares to fall almost 20%
421730925 THE NATION; GAO Report Faults Alaska Oil Cleanup Plans
419773993 White House panel suggests cloning research moratorium
432131990 Bush's Bioethics Advisory Panel Recommends a Moratorium, Not a Ban, on Cloning Research
432135066 Squaring Off Over Energy
398997464 Business Brief -- Cargill Inc.: Alliant Energy Agrees to Sell Its Share of 1997 Joint Venture
421728458 IN BRIEF / ENERGY; CMS Energy Gets New CFO, Sells Some Assets
419620922 CMS to move, cut jobs
409390530 A Dangerous Place for Women
408926250 Foreign athletes face more scrutiny ; Security checks, red tape more daunting since 9/11
408863897 Panel eager to assess 'where we are' ; Female members glad to be in Title IX group
405464560 POLITI

398875295 Prosecutors Demand Waksal Get A Seven-to-10-Year Prison Term
408844485 FCC may adopt 'opt-out' ; Telecoms could share call data
398859031 The Economy: Subprime Loans Will Be Covered By Federal Rules
432149726 Nicotine-Laced Water Is a Drug, F.D.A. Rules
421725731 FCC Rejects New Cell Phone Privacy Rules
421733873 FERC Raises Energy Price Caps; Power: Limit jumps to $250 per megawatt-hour under plan. Panel moves to cut Davis' authority over state electricity grid.
409306870 U.S. Regulators Tighten Policy on Problem Loans; Change to Cost Banks Millions in Charges
409280241 Scandals Haven't Hurt Much, Greenspan Says
421709108 California; US Home & Garden, 2 Units Sued for Weed Trimmer Safety Issues
432140359 Bristol-Myers Under Inquiry On Incentives for Drug Sales
432130685 Both Sides Say Bill Addressing Business Fraud Is a First Step
432131775 With By-the-Numbers Radio, Requests Are a Dying Breed
408882418 Hesitant states say no thanks to radiation pills
405467758 DOWNTOWN CROS

421737910 Davis Signals Emissions Bill Support; Politics: Governor 'inclined' to back measure targeting greenhouse gases.
432121986 McGreevey Issues Secret-Documents Order
408860323 Energy-market inquiries intensify
409373804 Ashcroft Assailed on Gun Policy Memo
432140028 Ross Perot Denies He Helped Manipulate Energy Market
421735703 The Region; Oxnard Recycling Firm Files for Chapter 11; Courts: Halaco seeks bankruptcy protection. But environmental groups say it's trying to avoid a pollution lawsuit.
432132842 S.E.C. Chief Rejects Calls for His Resignation
409330250 Justices' Recess: Calm Before the Storm
421918558 Health Net Lifts Freeze for New Medi-Cal Enrollees; Health care: The HMO reverses a policy that cut off access to Tenet doctors.
405469617 SPLIT DECISIONS: CAMPAIGN CONTRIBUTORS SPREAD WEALTH
432146212 Calls Increasing for Clarity on Food Labels
409313606 Firm Believer; With His Hollywood Management Company, Jeff Kwatinetz Is Reaching for the Stars
409312784 D.C. Council Ag

...

432135928 Regulators Allow Reliant Energy to Restructure
409288985 A Liability on the GOP Balance Sheet?; Halliburton Probe Is Growing Worry for Bush, Hill Republicans
421925601 The State; Simon's Aid to Parents; Election: Gubernatorial candidate proposes child-care tax breaks for parents, as well as for businesses.
421720698 Simon Campaigns in L.A.
432141837 Republican Lawmakers Seek Disclosures
432134078 Republicans Point Corporate-Abuse Finger at Democrats
432125093 Miners Are Rescued After 3 Days; 'We've Been Waiting,' They Say
432128806 White House Moves to Limit Corporate Scandals' Fallout
409291592 Nasdaq Acts To Tighten Rules on Companies
432143243 Anger at Executives' Profits Fuels Support for Stock Curb
398871741 Business Brief -- RHBT Financial Corp.: President Fired After Audit Shows Banking Irregularities
409305825 For Capital One Leaders, No Salary, Just Options
419747585 Governor gets heat on California tailpipe bill
405460007 BUSH'S RECORD ON CORPORATE ETHICS
421883572 

432122325 Andrx Gets Tentative F.D.A. Approval for Generic Drug
432124947 Agency Gets Green Help on River Work
421708914 THE NATION; Arrest of Pilots Put Focus on Abuse
398866009 The $5,000 Surgery Your Knees Don't Need; Rethinking `Scoping'
421732481 The State; Shortage of Preschools in Southland Grows; Access: Centers that boost school readiness are far more plentiful in Northern California, report finds. It says family finances suffer.
419687527 Nicor not sure all problems found ; More changes possible in firm's profit reports
419678758 Falun Gong jams official Chinese TV ; Brief broadcast of video begins 8-day intrusion
409294591 Banned Falun Gong Movement Jammed Chinese Satellite Signal
409298326 IBM to Pay $3.5 Billion For PwC Consulting; Accounting Firm Lacked Funds for Unit
432132062 Guns and Shields
398797375 California May Face Renewed Energy Crisis --- Signs of Trouble in State's Power Market Include Canceled Plants, Conservation Drop-Off
432120053 Ex-Counsel to Koch May Run

432125406 Blood Supply Dips in Region As New Rules Limit Donors
432140842 In a Switch, Utilities Say Power Is Low
419752901 Local phone calls revert to 5 cents ; Flat rate applies up to 15 miles
419622336 Senate OKs nuclear waste site in Nevada
408871353 Hot home sales ignite rush to get real estate licenses
432134257 World Briefing Americas: Canada: Province Won't Appeal Same-Sex Ruling
409377102 Mail-Order Molecules Brew a Terrorism Debate; Virus Created in Lab Raises Questions of Scrutiny for DNA Suppliers
432129348 Perot Systems Gets Official's Support
398877383 Pharmaceuticals Group Files Suit to Ban State Drug Lists --- Industry Worries Spread Of Such Systems May Hurt Pricing Power in the U.S.
432130985 Accord Suggests U.S. Prefers to Avoid Courts
421902909 The Nation; Ronald Brownstein WASHINGTON OUTLOOK; Political Free- Market Prosperity May Be Turning Corner
432116640 Many a Molehill Before Nuke Waste Finds Mountain
432128152 Forcing the Transit-Fare Issue, McCall Says Pataki 

421737386 THE NATION; Vehicle, Tire Makers Must Report on Fatalities
421742473 IN BRIEF/TOYS; Importer Fined Over Safety Violations
409295395 Senate Vote Spotlights Audit Reform And Sarbanes
421712158 THE NATION; Investors Not Easily Reassured; Wall Street: The day's wild swings cast doubt on U.S. leaders' ability to influence markets.
409308493 Stocks Rise As Profits Exceed Forecasts
421722866 A Power Struggle: Electric vs. Spiritual; Electricity: Indian tribes hope to block construction of a geothermal plant they fear will drain healing force of Medicine Lake.
432143682 Pacific Bell Agrees to a Fine To Settle D.S.L. Complaints
432140593 In Downtown Brooklyn, Too Tall Suddenly Fits
419739956 United, US Airways code plan draws fire
409394910 US Airways Offers U.S. An Equity Stake; Provision Part of Request For Loan Guarantee
432126497 Vanguard Airlines Files for Bankruptcy and Suspends Service
409313514 Restating, for the Record; Va. Ties for Highest Rate of Changes to Financial Statem

409306899 Frustrations Mount Over Clinton Airport's Closure; Hyde Field Pilots, Business Owners Seek Federal Relief
409373243 Frustration Lingers at Airport; At Still-Closed Clinton Site, Pilots, Business Owners Seek Relief
408871506 Bush seen above scandals' fray ; Public does not blame him for corporate crises
408872128 Bush seen above scandals' fray ; Public does not blame him for corporate crises
408928935 Watts plans to retire from House of Representatives ; Republican says work completed
408910047 Talks to proceed on corporate overhaul bill
408933217 Talks to proceed on corporate overhaul bill
408907496 Accounting lobbyists working overtime ; Congress looks at wider regulation
408931834 Scandal publicity drives accounting bill forward
408889873 Accounting lobbyists working overtime ; Congress looks at wider regulation
419747506 FDA calls nicotine water illegal drug, bars its sale ; Company sought to market it as a diet supplement
405465856 IN THE KITCHEN, TWO CAN BE A CROWD
41971